Generate learning data (Euler truncation errors) from random points on the phase space of the Van der Pol equation.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import h5py

In [ ]:
mu = 1.5
def vdp( t, x):
    y = np.empty(x.shape)
    y[0] = -mu*(x[1]*x[1]-1)*x[0]-x[1]
    y[1] = x[0]
    return y

In [ ]:
#this cell takes some time to run
np.random.seed(42) #fix seed for reproducibility
no = int(4e6) #number of data to generate

y_all = np.empty([no, 5])
y = np.array([0.0, 0.0])
for i in range(no):
    if i%1e5==0:
        print(i)
    y[0] = np.random.rand() * 10 - 5
    y[1] = np.random.rand() * 12 - 6
    dt = math.pow(10, np.random.rand()*3 - 3)
    rec_dt2 = 1/(dt*dt)
    sol = scipy.integrate.solve_ivp(vdp, [0, dt], y, t_eval = [dt], rtol=1e-10, atol=1e-10)
    dydt = vdp(0, y)
    y_all[i,0] = dt
    y_all[i,1] = y[0]
    y_all[i,2] = y[1]
    y_all[i,3] = rec_dt2*(sol.y[0] - y[0] - dt*dydt[0])
    y_all[i,4] = rec_dt2*(sol.y[1] - y[1] - dt*dydt[1])
print(y_all.shape)
y_all[428295,:]

Converting data and saving in hdf5 format

In [ ]:
path_to_hdf = 'data/vdp_data_ps_dt.hdf5'

In [ ]:
with h5py.File(path_to_hdf, 'a') as f:
    f.create_dataset(
        str('vdp_X'),
        data    = y_all[:,0:3],
        dtype   = np.float64,
        compression     = 'gzip',
        compression_opts= 8
        )
    f.create_dataset(
        str('vdp_Y'),
        data   = y_all[:,3:5],
        dtype  = np.float64,
        compression     = 'gzip',
        compression_opts= 8
        )


In [ ]:
f = h5py.File(path_to_hdf, 'r+')
X = f['vdp_X']
plt.figure(num="pl1")
plt.scatter(X[:,1], X[:,2]) 

In [ ]:
plt.figure(num="p123")
ax = plt.axes(projection ="3d")
ax.scatter3D(X[:,0], X[:,1], X[:,2], color = "green")
#plt.xscale('log')
#ax.view_init(0,60)
plt.show()

In [ ]:
f.close()